Берем результаты других ноутбуков и объединяем их тут

In [1]:
%cd ..

/workspace/TabDDPM_copy/TabDDPM


/venv/main/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import json
import pandas as pd
import torch
import matplotlib.pyplot as plt

from utils import *
from models.tabddpm_ohe_noise.tabddpm_ohe_noise import *

from tqdm.notebook import tqdm

from os import listdir
from os.path import isfile, join, isdir

from eval.base_metrics import calculate_base_metrics
from eval.similarity import calculate_similarity
from eval.mle import calculate_mle
from eval.alpha_beta import calculate_alpha_beta
from eval.detection import calculate_detection
from eval.dcr import calculate_DCR

In [3]:
datename = 'adult'

In [4]:
path = f'./eval/total/'
dirs = [join(path, f) for f in listdir(path) if isdir(join(path, f))]

In [5]:
results = []
for d in dirs:
    if datename in d:
        res = [pd.read_csv(join(d, f), header=[0, 1], skipinitialspace=True) for f in listdir(d) if isfile(join(d, f)) and 'csv' in f]
        results.extend(res)

results = pd.concat(results)
results = results.rename(columns={'Unnamed: 0_level_0':'', 
                                  'Unnamed: 1_level_0':'', 
                                  'Unnamed: 2_level_0':''}).drop(columns=[('base_metrics', '0')])

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
results.sort_values(('similarity', 'Overall Score (Average), %'), ascending=False).reset_index(drop=True)

\
              Model                             Type                Data   
0           TabDDPM                          tabddpm               adult   
1   tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_25  adult_ON_QnSC_200k   
2   tabddpm_ON_QnSC     tabddpm_ON_QnSC_200k_mult_10  adult_ON_QnSC_200k   
3   tabddpm_ON_QnSC           tabddpm_ON_QnSC_mult_5       adult_ON_QnSC   
4   tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_mult_6  adult_ON_QnSC_200k   
5   tabddpm_ON_QnSC  tabddpm_ON_QnSC_200k_const_0_01  adult_ON_QnSC_200k   
6   tabddpm_ON_QnSC       tabddpm_ON_QnSC_const_0_25       adult_ON_QnSC   
7   tabddpm_ON_QnSC           tabddpm_ON_QnSC_mult_6       adult_ON_QnSC   
8   tabddpm_ON_QnSC     tabddpm_ON_QnSC_200k_const_0  adult_ON_QnSC_200k   
9   tabddpm_ON_QnSC       tabddpm_ON_QnSC_const_0_01       adult_ON_QnSC   
10  tabddpm_ON_QnSC           tabddpm_ON_QnSC_mult_8       adult_ON_QnSC   
11  tabddpm_ON_QnSC           tabddpm_ON_QnSC_mult_7       adult_ON_QnSC   
12  tabddpm_ON_QnSC           tabddpm_ON_QnSC_mult_9       adult_ON_QnSC   
13  tabddpm_ON_QnSC      tabddpm_ON_QnSC_const_0_001       adult_ON_QnSC   
14  tabddpm_ON_QnSC          tabddpm_ON_QnSC_const_0       adult_ON_QnSC   
15  tabddpm_ON_QnSC      tabddpm_ON_QnSC_200k_mult_5  adult_ON_QnSC_200k   
16  tabddpm_ON_QnSC        tabddpm_ON_QnSC_const_0_5       adult_ON_QnSC   
17  tabddpm_ON_QnSC        tabddpm_ON_QnSC_const_0_1       adult_ON_QnSC   
18  tabddpm_ON_QnSC           tabddpm_ON_QnSC_mult_3       adult_ON_QnSC   
19  tabddpm_ON_QnSC           tabddpm_ON_QnSC_mult_2       adult_ON_QnSC   
20  tabddpm_ON_QnSC           tabddpm_ON_QnSC_mult_1       adult_ON_QnSC   
21   tabddpm_ON_QnI       tabddpm_ON_QnI_const_0_001        adult_ON_QnI   
22   tabddpm_ON_QnI           tabddpm_ON_QnI_const_0        adult_ON_QnI   
23  tabddpm_ON_QnSC           tabddpm_ON_QnSC_mult_0       adult_ON_QnSC   
24       TabDDPM ON       tabddpm_ON_SC_NA_const_0_5      adult_ON_SC_NA   
25       TabDDPM ON          tabddpm_ON_SC_NA_mult_0      adult_ON_SC_NA   
26       TabDDPM ON          tabddpm_ON_SC_NA_mult_8      adult_ON_SC_NA   
27       TabDDPM ON          tabddpm_ON_SC_NA_mult_9      adult_ON_SC_NA   
28       TabDDPM ON            tabddpm_ON_SC_const_0         adult_ON_SC   
29       TabDDPM ON        tabddpm_ON_SC_const_0_001         adult_ON_SC   
30       TabDDPM ON          tabddpm_ON_SC_NA_mult_4      adult_ON_SC_NA   
31       TabDDPM ON          tabddpm_ON_SC_NA_mult_7      adult_ON_SC_NA   
32       TabDDPM ON     tabddpm_ON_SC_NA_const_0_001      adult_ON_SC_NA   
33       TabDDPM ON          tabddpm_ON_SC_NA_mult_5      adult_ON_SC_NA   
34       TabDDPM ON       tabddpm_ON_SC_NA_const_0_1      adult_ON_SC_NA   
35       TabDDPM ON      tabddpm_ON_SC_NA_const_0_25      adult_ON_SC_NA   
36       TabDDPM ON          tabddpm_ON_SC_NA_mult_2      adult_ON_SC_NA   
37       TabDDPM ON          tabddpm_ON_SC_NA_mult_6      adult_ON_SC_NA   
38       TabDDPM ON         tabddpm_ON_SC_NA_const_0      adult_ON_SC_NA   
39       TabDDPM ON          tabddpm_ON_SC_NA_mult_1      adult_ON_SC_NA   
40       TabDDPM ON      tabddpm_ON_SC_NA_const_0_01      adult_ON_SC_NA   
41       TabDDPM ON          tabddpm_ON_SC_NA_mult_3      adult_ON_SC_NA   
42       TabDDPM ON             tabddpm_ON_SC_mult_2         adult_ON_SC   
43       TabDDPM ON             tabddpm_ON_SC_mult_0         adult_ON_SC   
44       TabDDPM ON          tabddpm_ON_SC_const_0_1         adult_ON_SC   
45       TabDDPM ON         tabddpm_ON_SC_const_0_25         adult_ON_SC   
46       TabDDPM ON             tabddpm_ON_SC_mult_7         adult_ON_SC   
47       TabDDPM ON             tabddpm_ON_SC_mult_4         adult_ON_SC   
48       TabDDPM ON                tabddpm_ON_mult_3            adult_ON   
49       TabDDPM ON          tabddpm_ON_SC_const_0_5         adult_ON_SC   
50       TabDDPM ON             tabddpm_ON_SC_mult_3         adult_ON_SC   
51       TabDDPM ON             tabddpm_ON_SC_